In [1]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import numpy as np
import pandas as pd
import glob,imp,os,sys
from pandas import json_normalize
import pyslha
sys.path.append('/home/camila/smodels/')
from smodels.share.models.mssm import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.base.model import Model
from smodels.particlesLoader import getParticlesFromSLHA

pd.options.mode.chained_assignment = None #Disable copy warnings

/tmp/ipykernel_251010/3423779146.py:5: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import glob,imp,os,sys


In [2]:
BSMlist = getParticlesFromSLHA('/home/camila/MonoXSMS/slha_files_pseudoscalar_0j1j_scan/run_01_pseudoscalar_100_1.slha')
model = Model(BSMparticles=BSMlist, SMparticles=SMList)
print(BSMlist)

[gha, gha~, ghz, ghz~, ghwp, ghwp~, ghwm, ghwm~, ghg, ghg~, xr, xc, xc~, xd, xd~, y0]


In [3]:
slhaFolder = '/home/camila/MonoXSMS/slha_files_pseudoscalar_0j1j_scan/'

In [4]:
slhaData = []
for f in glob.glob(slhaFolder+'/*.slha'):
    slhaFile = f
    slha = pyslha.readSLHAFile(slhaFile)
    massDict = dict([[str(key),abs(val)] for key,val in slha.blocks['MASS'].items() if key >= 52])
    extparDict = dict([[str(key),val] for key,val in slha.blocks['DMINPUTS'].items()])
#     extparDict = dict([[str(key),val] for key,val in slha.blocks['ZPRIME'].items()])
#     extparDict.update(dict([[str(key+len(extparDict)),val] for key,val in slha.blocks['BLINPUTS'].items()]))
    
    widthDict = dict([[str(key),val.totalwidth] for key,val in slha.decays.items() if key >= 52])
    BRsDict = {}
#     for pdg,val in slha.decays.items():
# #         if not abs(pdg) in [1000024,1000023]:
# #             continue
#         initialState = model.getParticlesWith(pdg=pdg)[0].label            
#         BRsDict[initialState] = {}
#         for dec in val.decays:
#             if dec.br < 0.0001: continue            
#             finalState = ','.join([model.getParticlesWith(pdg=pid)[0].label for pid in sorted(dec.ids)])
#             BRsDict[initialState][finalState] = dec.br
    xsec8TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=8000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=8000)])
    xsec13TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=13000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=13000)])    
    slhaDict = {'filename' : f.split('/')[-1], 'mass' : massDict, 'width' : widthDict, 'extpar' : extparDict, 
                'xsec8TeV(fb)' : xsec8TeV, 'xsec13TeV(fb)' : xsec13TeV, 'BRs' : BRsDict}
    slhaData.append(slhaDict)

In [5]:
#Convert to DataFrame
slhaDF = json_normalize(slhaData)
#Add total cross-sections:
xsecs13 = [x for x in list(slhaDF) if 'xsec13TeV' in x]
xsecs8 = [x for x in list(slhaDF) if 'xsec8TeV' in x]
slhaDF['totalxsec13TeV(fb)'] = slhaDF[xsecs13].sum(axis=1)
slhaDF['totalxsec8TeV(fb)'] = slhaDF[xsecs8].sum(axis=1)

In [6]:
slhaDF

,filename,mass.52,mass.54,mass.5000001,mass.9000002,mass.9000003,mass.9000004,width.52,width.54,width.5000001,...,extpar.19,extpar.20,extpar.21,extpar.22,extpar.23,extpar.24,extpar.25,xsec13TeV(fb).54_54,totalxsec13TeV(fb),totalxsec8TeV(fb)
0,run_02_pseudoscalar_150_1.slha,1.0,150.0,10.0,91.1876,79.82436,79.82436,0.0,5.967780,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24855.000,24855.000,0.0
1,run_40_pseudoscalar_750_200.slha,200.0,750.0,10.0,91.1876,79.82436,79.82436,0.0,64.063640,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,383.570,383.570,0.0
2,run_60_pseudoscalar_850_300.slha,300.0,850.0,10.0,91.1876,79.82436,79.82436,0.0,69.231890,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,186.600,186.600,0.0
3,run_71_pseudoscalar_900_400.slha,400.0,900.0,10.0,91.1876,79.82436,79.82436,0.0,64.849010,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,133.790,133.790,0.0
4,run_49_pseudoscalar_700_250.slha,250.0,700.0,10.0,91.1876,79.82436,79.82436,0.0,55.003580,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,570.290,570.290,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,run_04_pseudoscalar_250_1.slha,1.0,250.0,10.0,91.1876,79.82436,79.82436,0.0,9.946866,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12080.000,12080.000,0.0
69,run_13_pseudoscalar_700_1.slha,1.0,700.0,10.0,91.1876,79.82436,79.82436,0.0,63.363160,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,568.850,568.850,0.0
70,run_69_pseudoscalar_1000_350.slha,350.0,1000.0,10.0,91.1876,79.82436,79.82436,0.0,83.109030,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.116,70.116,0.0
71,run_55_pseudoscalar_1000_250.slha,250.0,1000.0,10.0,91.1876,79.82436,79.82436,0.0,89.152240,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.137,70.137,0.0


In [7]:
slhaDF = slhaDF.rename(columns={'mass.52': '$m_{DM}$', 'mass.54': '$m_{med}$'})

In [12]:
slhaDF = slhaDF.rename(columns={'totalxsec13TeV(fb)': 'Total xsec (pb)', 'xsec13TeV(fb).54_54': 'Total xsec-pT150 (pb)'})

In [9]:
slhaDF['Total xsec (pb)'] = slhaDF['Total xsec (pb)']/1e3

In [13]:
slhaDF

,filename,$m_{DM}$,$m_{med}$,mass.5000001,mass.9000002,mass.9000003,mass.9000004,width.52,width.54,width.5000001,...,extpar.19,extpar.20,extpar.21,extpar.22,extpar.23,extpar.24,extpar.25,Total xsec-pT150 (pb),Total xsec (pb),totalxsec8TeV(fb)
0,run_02_pseudoscalar_150_1.slha,1.0,150.0,10.0,91.1876,79.82436,79.82436,0.0,5.967780,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24855.000,24.855000,0.0
1,run_40_pseudoscalar_750_200.slha,200.0,750.0,10.0,91.1876,79.82436,79.82436,0.0,64.063640,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,383.570,0.383570,0.0
2,run_60_pseudoscalar_850_300.slha,300.0,850.0,10.0,91.1876,79.82436,79.82436,0.0,69.231890,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,186.600,0.186600,0.0
3,run_71_pseudoscalar_900_400.slha,400.0,900.0,10.0,91.1876,79.82436,79.82436,0.0,64.849010,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,133.790,0.133790,0.0
4,run_49_pseudoscalar_700_250.slha,250.0,700.0,10.0,91.1876,79.82436,79.82436,0.0,55.003580,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,570.290,0.570290,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,run_04_pseudoscalar_250_1.slha,1.0,250.0,10.0,91.1876,79.82436,79.82436,0.0,9.946866,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12080.000,12.080000,0.0
69,run_13_pseudoscalar_700_1.slha,1.0,700.0,10.0,91.1876,79.82436,79.82436,0.0,63.363160,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,568.850,0.568850,0.0
70,run_69_pseudoscalar_1000_350.slha,350.0,1000.0,10.0,91.1876,79.82436,79.82436,0.0,83.109030,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.116,0.070116,0.0
71,run_55_pseudoscalar_1000_250.slha,250.0,1000.0,10.0,91.1876,79.82436,79.82436,0.0,89.152240,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.137,0.070137,0.0


In [14]:
slhaDF['Total xsec-pT150 (pb)'] = slhaDF['Total xsec-pT150 (pb)']/1e3

In [15]:
slhaDF = slhaDF[['$m_{DM}$', '$m_{med}$', 'Total xsec (pb)', 'Total xsec-pT150 (pb)']]

In [16]:
slhaDF.sort_values(['$m_{DM}$', '$m_{med}$'], inplace=True)

In [18]:
#Save DataFrame to pickle file:
slhaDF.to_pickle('/home/camila/MonoXSMS/slha_files_pseudoscalar_0j1j_scan/xsecs0j1j.pcl')

In [18]:
f.split('/')[-2]+'/'+f.split('/')[-1]

'05/run_18_axial_2000_1.slha'

In [8]:
0.1017/0.1094

0.9296160877513712